## Confusion matrix for Irrigated Fields

  - In a meeting (May 19th) we decided to drop non-irrigated fields.
  - NASS has to stay in (do not filter it).
  - Perennials and Alfalfa has to go out, i.e. Keep those that are potentially double-cropped.
  

In [1]:
import csv
import numpy as np
import pandas as pd

# import geopandas as gpd
from IPython.display import Image
# from shapely.geometry import Point, Polygon

from math import factorial
import datetime
import time
import scipy
import os, os.path
import itertools
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.linear_model import LinearRegression
from patsy import cr

# from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sb


import sys
start_time = time.time()

/Users/hn/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
sys.path.append('/Users/hn/Documents/00_GitHub/Ag/remote_sensing/python/')
sys.path.append('/home/hnoorazar/remote_sensing_codes/')

import remote_sensing_core as rc
import remote_sensing_core as rcp

In [3]:
param_dir = "/Users/hn/Documents/00_GitHub/Ag/remote_sensing/parameters/"

double_crop_potens = pd.read_csv(param_dir + "double_crop_potential_plants.csv")

#### Read ShapeFile Data

In [6]:
# We may need to write a for-loop if we want more than one county.
SF_year = 2016
given_county = "Yakima"
indeks = "EVI"
raw_or_regular = "regular"

In [7]:
shapeFile_Data_dir = "/Users/hn/Documents/01_research_data/" + \
                      "remote_sensing/01_Data_part_not_filtered/"

WSDA_DataTable = pd.read_csv(shapeFile_Data_dir + "WSDA_DataTable_" + str(SF_year) + ".csv")

WSDA_DataTable = WSDA_DataTable[WSDA_DataTable.county == given_county]

WSDA_DataTable["DataSrc"] = WSDA_DataTable["DataSrc"].str.lower()
WSDA_DataTable["CropTyp"] = WSDA_DataTable["CropTyp"].str.lower()

### Generate different datafamres based on the following variables

- 2 * 2 * 2 * 2 different combinations of NASS, Double_by_Notes, Irrigated, LastSurveyYear

For now, leave out the fucking last survey year!

In the following we have abbreviated:  
 - AF: All Fields
 - DP: Double Potential Fields (i.e. perennials out)
 - Irr: Just Irrigated Fields
 - BothIrr: both irrigated and non-irrigated

In [8]:
data_dir = "/Users/hn/Documents/01_research_data/remote_sensing/test_aeolus_outputs/" + \
           "04_noJump_Regularized_plt_tbl_SOSEOS/"

In [9]:
SG_win_size = 5
SG_order = 1

In [ ]:
f_name = given_county  + "_" + str(SF_year) + "_regular_" + indeks + \
         "_SG_win" + str(SG_win_size) + "_Order" + str(SG_order) + ".csv"
a_df = pd.read_csv(data_dir + f_name, low_memory=False)

# Inside the for-loop

In [20]:
deltas = [0.1, 0.2, 0.3, 0.4]
windows_degrees = [[3, 1], [3, 2],
                   [5, 1], [5, 2], [5, 3],
                   [7, 1], [7, 2], [7, 3], 
                   [9, 1], [9, 2], [9, 3]]

# delta_windows_degrees = [[i, j] for i in deltas for j in windows_degrees]
delta_windows_degrees = [[5, 1], [5, 3], [7, 3], [9, 3]]

In [21]:
output_columns = ['parameters', 'actual_2_pred_2', 'actual_2_pred_Not2',
                  'actual_Not2_pred_2', 'actual_Not2_pred_Not2',]

In [22]:
exactly_2_peaks = False
double_by_Note = False
NASS_out = False
non_Irr_out = True
perennials_out = True

In [23]:
output = pd.DataFrame(data=None, 
                      #delta_windows_degrees
                      index = np.arange(len(delta_windows_degrees)),
                      columns = output_columns)
output['parameters'] = delta_windows_degrees

#### 
#### Build shapeFile info accordingly
#### 
curr_SF = WSDA_DataTable.copy()

if double_by_Note == False:
    dbl_name = "_dblNotFiltered_"
else:
    curr_SF = rc.filter_double_by_Notes(curr_SF)
    dbl_name = "_onlyDblByNotes_"

if NASS_out == True:
    curr_SF = rc.filter_out_NASS(curr_SF)
    NASS_name = "NASSOut_"
else:
    NASS_name = "NASSin_"

if non_Irr_out == True:
    curr_SF = rc.filter_out_nonIrrigated(curr_SF)
    non_Irr_name = "JustIrr"
else:
    non_Irr_name = "BothIrr"

if perennials_out == True:
    curr_SF = curr_SF[curr_SF.CropTyp.isin(double_crop_potens['Crop_Type'])]
    Pere_name = "_PereOut_"
else:
    Pere_name = "_PereIn_"


print ("NASS_out: " + str(NASS_out) + ", non_Irr_out: " + str(non_Irr_out) + \
       ", perennials_out: " + str(perennials_out))

NASS_out: False, non_Irr_out: True, perennials_out: True


In [31]:
doubl_pk_file = data_dir + f_name
doubl_peak_table = pd.read_csv( doubl_pk_file, low_memory=False)

In [33]:
doubl_peak_table.tail(2)

,ID,Acres,county,CropGrp,CropTyp,DataSrc,ExctAcr,IntlSrD,Irrigtn,LstSrvD,...,image_year,SF_year,doy,EVI,human_system_start_time,Date,system_start_time,EVI_ratio,SOS,EOS
20033,49382_WSDA_SF_2016,5,Yakima,Other,pasture,wsda,5.474619,2016/11/08 00:00:00,flood,2016/11/08 00:00:00,...,2016,2016,1,-0.415945,2016-01-01 00:00:00,2016-01-01 00:00:00,1451689400000,666.0,666.0,666.0
20034,39201_WSDA_SF_2016,7,Yakima,Hay/Silage,grass hay,wsda,7.193052,2006/03/15 00:00:00,wheel line,2016/11/08 00:00:00,...,2016,2016,1,-0.475781,2016-01-01 00:00:00,2016-01-01 00:00:00,1451689400000,0.0,-0.0,-0.0


In [34]:
doubl_peak_table["CropTyp"] = doubl_peak_table["CropTyp"].str.lower()

In [37]:
if double_by_Note == True:
    doubl_peak_table = rc.filter_double_by_Notes(doubl_peak_table)

if NASS_out == True:
    doubl_peak_table = rc.filter_out_NASS(doubl_peak_table)

In [38]:
if non_Irr_out == True:
    doubl_peak_table = rc.filter_out_nonIrrigated(doubl_peak_table)

In [40]:
if perennials_out == True:
    doubl_peak_table = doubl_peak_table[\
             doubl_peak_table.CropTyp.isin(double_crop_potens['Crop_Type'])]

In [41]:
doubl_peak_table.shape

(3179, 25)

In [42]:
doubl_peak_table.drop(['max_Doy', 'max_value'], axis=1, inplace=True)
doubl_peak_table.drop_duplicates(inplace=True)
doubl_peak_table.shape

(3179, 25)

In [44]:
doubl_peak_table.head(10)

,ID,Acres,county,CropGrp,CropTyp,DataSrc,ExctAcr,IntlSrD,Irrigtn,LstSrvD,...,image_year,SF_year,doy,EVI,human_system_start_time,Date,system_start_time,EVI_ratio,SOS,EOS
25,41582_WSDA_SF_2016,1,Yakima,Cereal Grain,barley,wsda,0.500940,2005/11/01 00:00:00,wheel line,2012/11/06 00:00:00,...,2016,2016,1,-0.010833,2016-01-01 00:00:00,2016-01-01 00:00:00,1451689400000,0.021136,-0.000000,-0.000000
29,47674_WSDA_SF_2016,7,Yakima,Cereal Grain,barley,wsda,6.605969,2009/03/25 00:00:00,wheel line,2012/11/07 00:00:00,...,2016,2016,1,-0.155522,2016-01-01 00:00:00,2016-01-01 00:00:00,1451689400000,666.000000,666.000000,666.000000
30,43600_WSDA_SF_2016,10,Yakima,Cereal Grain,barley,wsda,10.223197,2012/11/07 00:00:00,wheel line,2012/11/07 00:00:00,...,2016,2016,1,-0.211498,2016-01-01 00:00:00,2016-01-01 00:00:00,1451689400000,0.016490,-0.000000,-0.000000
31,41016_WSDA_SF_2016,6,Yakima,Cereal Grain,oat,wsda,5.673938,2005/11/01 00:00:00,sprinkler,2012/11/27 00:00:00,...,2016,2016,1,0.315063,2016-01-01 00:00:00,2016-01-01 00:00:00,1451689400000,0.410890,0.000000,0.000000
42,35468_WSDA_SF_2016,10,Yakima,Hay/Silage,barley hay,wsda,9.892750,2002/11/15 00:00:00,wheel line,2012/11/27 00:00:00,...,2016,2016,1,-0.278018,2016-01-01 00:00:00,2016-01-01 00:00:00,1451689400000,0.019667,-0.000000,-0.000000
162,36170_WSDA_SF_2016,38,Yakima,Hay/Silage,sudangrass,wsda,37.557524,2002/05/28 00:00:00,center pivot,2015/04/23 00:00:00,...,2016,2016,61,0.412277,2016-03-01 00:00:00,2016-03-01 00:00:00,1456873400000,0.536208,0.412277,0.000000
163,36170_WSDA_SF_2016,38,Yakima,Hay/Silage,sudangrass,wsda,37.557524,2002/05/28 00:00:00,center pivot,2015/04/23 00:00:00,...,2016,2016,101,0.343879,2016-04-10 00:00:00,2016-04-10 00:00:00,1460329400000,0.455658,0.000000,0.343879
164,36170_WSDA_SF_2016,38,Yakima,Hay/Silage,sudangrass,wsda,37.557524,2002/05/28 00:00:00,center pivot,2015/04/23 00:00:00,...,2016,2016,181,0.501776,2016-06-29 00:00:00,2016-06-29 00:00:00,1467241400000,0.641610,0.501776,0.000000
165,36170_WSDA_SF_2016,38,Yakima,Hay/Silage,sudangrass,wsda,37.557524,2002/05/28 00:00:00,center pivot,2015/04/23 00:00:00,...,2016,2016,281,0.361515,2016-10-07 00:00:00,2016-10-07 00:00:00,1475881400000,0.476427,0.000000,0.361515
168,48197_WSDA_SF_2016,9,Yakima,Cereal Grain,"corn, field",wsda,9.216289,2002/05/28 00:00:00,rill,2015/04/23 00:00:00,...,2016,2016,1,0.080020,2016-01-01 00:00:00,2016-01-01 00:00:00,1451689400000,0.123637,0.000000,0.000000


In [ ]:
for indeks in ["EVI"]:

    ####
    ####  parameters
    ####
    deltas = [0.1, 0.2, 0.3, 0.4]
    windows_degrees = [[3, 1], [3, 2],
                       [5, 1], [5, 2], [5, 3],
                       [7, 1], [7, 2], [7, 3], 
                       [9, 1], [9, 2], [9, 3]]

    # delta_windows_degrees = [[i, j] for i in deltas for j in windows_degrees]
    delta_windows_degrees = [[5, 1], [5, 3], [7, 3], [9, 3]]

    output_columns = ['parameters', 'actual_2_pred_2', 'actual_2_pred_Not2',
                      'actual_Not2_pred_2', 'actual_Not2_pred_Not2',]

    for exactly_2_peaks in [False, True]:
        for double_by_Note in [False]:
            for NASS_out in [False]:
                # we have dropped out non-irrigated fields in the peak_finding step
                for non_Irr_out in [True]: 
                    for perennials_out in [True]: # 
                        #### 
                        #### build output dataframe
                        #### 
                        output = pd.DataFrame(data=None, 
                                              #delta_windows_degrees
                                              index = np.arange(len(delta_windows_degrees)),
                                              columns = output_columns)
                        output['parameters'] = delta_windows_degrees

                        #### 
                        #### Build shapeFile info accordingly
                        #### 
                        curr_SF = WSDA_DataTable.copy()

                        if double_by_Note == False:
                            dbl_name = "_dblNotFiltered_"
                        else:
                            curr_SF = rc.filter_double_by_Notes(curr_SF)
                            dbl_name = "_onlyDblByNotes_"

                        if NASS_out == True:
                            curr_SF = rc.filter_out_NASS(curr_SF)
                            NASS_name = "NASSOut_"
                        else:
                            NASS_name = "NASSin_"

                        if non_Irr_out == True:
                            curr_SF = rc.filter_out_nonIrrigated(curr_SF)
                            non_Irr_name = "JustIrr"
                        else:
                            non_Irr_name = "BothIrr"

                        if perennials_out == True:
                            curr_SF = curr_SF[curr_SF.CropTyp.isin(double_crop_potens['Crop_Type'])]
                            Pere_name = "_PereOut_"
                        else:
                            Pere_name = "_PereIn_"


                        print ("NASS_out: " + str(NASS_out) + ", non_Irr_out: " + str(non_Irr_out) + \
                               ", perennials_out: " + str(perennials_out))

                        for location, params in enumerate(output['parameters']):
            #                 if location % 15 ==0:
            #                     print (location)
                            window = params[0]
                            degree = params[1]
                            
                            f_name = given_county  + "_" + str(SF_year) + "_regular_" + indeks + \
                                     "_SG_win" + str(SG_win_size) + "_Order" + str(SG_order) + ".csv"
                            
                            doubl_pk_file = data_dir + f_name

                            doubl_peak_table = pd.read_csv( doubl_pk_file, low_memory=False)

                            doubl_peak_table["CropTyp"] = doubl_peak_table["CropTyp"].str.lower()

                            if double_by_Note == True:
                                doubl_peak_table = rc.filter_double_by_Notes(doubl_peak_table)

                            if NASS_out == True:
                                doubl_peak_table = rc.filter_out_NASS(doubl_peak_table)

                             # print ("NASS_out: " + str(NASS_out))
                             # print ("curr_SF.DataSrc.unique(): ")
                             # print (curr_SF.DataSrc.unique())

                             # print ("doubl_peak_table.DataSrc.unique(): ")
                             # print (doubl_peak_table.DataSrc.unique())
                             # print("____________________________________________________")

                            if non_Irr_out == True:
                                doubl_peak_table = rc.filter_out_nonIrrigated(doubl_peak_table)
                            # print("_____________________________________________")
                            # print("non_Irr_out " + str(non_Irr_out) + " doubl_peak_table.Irrigtn:")
                            # print (doubl_peak_table.Irrigtn.unique())

                            if perennials_out == True:
                                doubl_peak_table = doubl_peak_table[\
                                         doubl_peak_table.CropTyp.isin(double_crop_potens['Crop_Type'])]
                             # print("perennials_out " + str(perennials_out))
                             # print (doubl_peak_table.CropTyp.unique())

                             # print ("No. of rows after dropping perennials \
                             #          are [%(nrow)d]." % {"nrow":doubl_peak_table.shape[0]})

                            # print ("No. of rows are [%(nrow)d]." % {"nrow":doubl_peak_table.shape[0]})
                            doubl_peak_table.drop(['max_Doy', 'max_value'], axis=1, inplace=True)
                            doubl_peak_table.drop_duplicates(inplace=True)
                            # print ("No. of rows after dropping dupliates are [%(nrow)d]." % \
                            # {"nrow":doubl_peak_table.shape[0]})

                            #### 
                            #### Populate output dataframe
                            #### 
                            actual_double_cropped = rc.filter_double_by_Notes(curr_SF)
                            actual_Notdouble_cropped = rc.filter_Notdouble_by_Notes(curr_SF)

                            if exactly_2_peaks == False:
                                predicted_double_peaks = doubl_peak_table[\
                                                         doubl_peak_table.max_count >= 2].copy()
                                
                                predicted_Notdouble_peaks = doubl_peak_table[\
                                                            doubl_peak_table.max_count < 2].copy()
                                exactly_2_peaks_name = "morethan2Peaks"
                            else:
                                predicted_double_peaks = doubl_peak_table[\
                                                                    doubl_peak_table.max_count == 2].copy()
                                
                                
                                predicted_Notdouble_peaks = doubl_peak_table[\
                                                            doubl_peak_table.max_count != 2].copy()
                                
                                exactly_2_peaks_name = "exactly2Peaks"

                            # print ("There are [%(nrow)d] IDs in curr_SF." % \
                            #       {"nrow":len(curr_SF['ID'])})

                            # print ("of which [%(nrow)d] are unique." % \
                            #        {"nrow":len(curr_SF['ID'].unique())})

                            actual_2_pred_2 = actual_double_cropped[\
                                            actual_double_cropped['ID'].isin(predicted_double_peaks['ID'])]

                            actual_2_pred_2 = actual_2_pred_2['ExctAcr'].sum()

                            actual_Not2_pred_2 = actual_Notdouble_cropped[\
                                                   actual_Notdouble_cropped['ID'].isin(\
                                                                  predicted_double_peaks['ID'])]
                            actual_Not2_pred_2 = actual_Not2_pred_2['ExctAcr'].sum()

                            # the follwoing two lines would not work, since we have not 
                            # included the fields with no maximum in our output
                            # actual_2_pred_Not2 = sum(actual_double_cropped['ID'].isin(\
                            #                                  predicted_Notdouble_peaks['ID']))
                            # actual_Not2_pred_Not2 = sum(actual_Notdouble_cropped['ID'].isin(\
                            #                            predicted_Notdouble_peaks['ID']))

                            actual_2_pred_Not2 = actual_double_cropped['ExctAcr'].sum() - actual_2_pred_2

                            actual_Not2_pred_Not2 = actual_Notdouble_cropped['ExctAcr'].sum() - \
                                                      actual_Not2_pred_2

                            fillin_col = ["actual_2_pred_2", "actual_2_pred_Not2", \
                                          "actual_Not2_pred_2", "actual_Not2_pred_Not2"]

                            fillin_vals = [actual_2_pred_2, actual_2_pred_Not2, \
                                           actual_Not2_pred_2, actual_Not2_pred_Not2]

                            output.loc[location, fillin_col] = fillin_vals


                        ###########
                        output['parameters'] = output['parameters'].astype("str")
                        write_path = "/Users/hn/Documents/01_research_data/" + \
                                     "remote_sensing/01_NDVI_TS/00_Eastern_WA_withYear/2Years/" + \
                                     "2Yrs_tables_" + raw_or_regular + "/"
                                      

                        os.makedirs(write_path, exist_ok=True)
                        filename = write_path + given_county + "_" + str(SF_year) + "_" + indeks + \
                                   Pere_name + NASS_name + non_Irr_name + dbl_name + \
                                   "confusion_Acr_" + exactly_2_peaks_name + "_" + \
                                    raw_or_regular + ".csv"


                        output['actual_2_pred_2'] = output['actual_2_pred_2'].astype(float)
                        output['actual_2_pred_Not2'] = output['actual_2_pred_Not2'].astype(float)
                        output['actual_Not2_pred_2'] = output['actual_Not2_pred_2'].astype(float)
                        output['actual_Not2_pred_Not2'] = output['actual_Not2_pred_Not2'].astype(float)
                        output = output.round(decimals=2)

                        output.to_csv(filename, index = False)